## LBC (London stream)
http://media-ice.musicradio.com/LBCLondon.m3u (HE AAC format)  
http://media-ice.musicradio.com/LBCLondonMP3Low (MP3 low bit rate)

## LBC (National stream)
http://media-ice.musicradio.com/LBCUK.m3u (HE AAC format)  
http://media-ice.musicradio.com/LBCUKMP3Low (MP3 low bit rate)

## LBC London News 1152
http://media-ice.musicradio.com/LBC1152.m3u (HE AAC format)  
http://media-ice.musicradio.com/LBC1152MP3Low (MP3 low bit rate)

In [ ]:
import requests

In [ ]:
stream_url = 'http://media-ice.musicradio.com/LBCLondonMP3Low'

r = requests.get(stream_url, stream=True)

with open('stream.mp3', 'wb') as f:
    try:
        for block in r.iter_content(1024):
            f.write(block)
    except KeyboardInterrupt:
        pass

## Converting mp3 to wav

In [1]:
import subprocess

In [ ]:
subprocess.call(['track2track -t wav blind-spots-in-reinforcement-learning.mp3 --sample-rate=16000 --channels=1'], shell=True)

## Splitting long wav into sentence chunks

In [2]:
import collections
import contextlib
import sys
import wave
import os

import webrtcvad

In [40]:
def read_wave(path):
    """Reads a .wav file.
    Takes the path, and returns (PCM audio data, sample rate).
    """
    with contextlib.closing(wave.open(path, 'rb')) as wf:
        num_channels = wf.getnchannels()
        assert num_channels == 1
        sample_width = wf.getsampwidth()
        assert sample_width == 2
        sample_rate = wf.getframerate()
        assert sample_rate in (8000, 16000, 32000)
        pcm_data = wf.readframes(wf.getnframes())
    
    return pcm_data, sample_rate

def write_wave(path, audio, sample_rate):
    """Writes a .wav file.

    Takes path, PCM audio data, and sample rate.
    """
    with contextlib.closing(wave.open(path, 'wb')) as wf:
        wf.setnchannels(1)
        wf.setsampwidth(2)
        wf.setframerate(sample_rate)
        wf.writeframes(audio)


class Frame(object):
    """Represents a "frame" of audio data."""
    def __init__(self, bytes, timestamp, duration):
        self.bytes = bytes
        self.timestamp = timestamp
        self.duration = duration


def frame_generator(frame_duration_ms, audio, sample_rate):
    """Generates audio frames from PCM audio data.

    Takes the desired frame duration in milliseconds, the PCM data, and
    the sample rate.

    Yields Frames of the requested duration.
    """
    n = int(sample_rate * (frame_duration_ms / 1000.0) * 2)
    offset = 0
    timestamp = 0.0
    duration = (float(n) / sample_rate) / 2.0
    while offset + n < len(audio):
        yield Frame(audio[offset:offset + n], timestamp, duration)
        timestamp += duration
        offset += n


def vad_collector(sample_rate, frame_duration_ms,
                  padding_duration_ms, vad, frames):
    """Filters out non-voiced audio frames.

    Given a webrtcvad.Vad and a source of audio frames, yields only
    the voiced audio.

    Uses a padded, sliding window algorithm over the audio frames.
    When more than 90% of the frames in the window are voiced (as
    reported by the VAD), the collector triggers and begins yielding
    audio frames. Then the collector waits until 90% of the frames in
    the window are unvoiced to detrigger.

    The window is padded at the front and back to provide a small
    amount of silence or the beginnings/endings of speech around the
    voiced frames.

    Arguments:

    sample_rate - The audio sample rate, in Hz.
    frame_duration_ms - The frame duration in milliseconds.
    padding_duration_ms - The amount to pad the window, in milliseconds.
    vad - An instance of webrtcvad.Vad.
    frames - a source of audio frames (sequence or generator).

    Returns: A generator that yields PCM audio data.
    """
    num_padding_frames = int(padding_duration_ms / frame_duration_ms)
    # We use a deque for our sliding window/ring buffer.
    ring_buffer = collections.deque(maxlen=num_padding_frames)
    # We have two states: TRIGGERED and NOTTRIGGERED. We start in the
    # NOTTRIGGERED state.
    triggered = False

    voiced_frames = []
    for frame in frames:
        is_speech = vad.is_speech(frame.bytes, sample_rate)

        sys.stdout.write('1' if is_speech else '0')
        if not triggered:
            ring_buffer.append((frame, is_speech))
            num_voiced = len([f for f, speech in ring_buffer if speech])
            # If we're NOTTRIGGERED and more than 90% of the frames in
            # the ring buffer are voiced frames, then enter the
            # TRIGGERED state.
            if num_voiced > 0.9 * ring_buffer.maxlen:
                triggered = True
                sys.stdout.write('+(%s)' % (ring_buffer[0][0].timestamp,))
                # We want to yield all the audio we see from now until
                # we are NOTTRIGGERED, but we have to start with the
                # audio that's already in the ring buffer.
                for f, s in ring_buffer:
                    voiced_frames.append(f)
                ring_buffer.clear()
        else:
            # We're in the TRIGGERED state, so collect the audio data
            # and add it to the ring buffer.
            voiced_frames.append(frame)
            ring_buffer.append((frame, is_speech))
            num_unvoiced = len([f for f, speech in ring_buffer if not speech])
            # If more than 90% of the frames in the ring buffer are
            # unvoiced, then enter NOTTRIGGERED and yield whatever
            # audio we've collected.
            if num_unvoiced > 0.9 * ring_buffer.maxlen:
                sys.stdout.write('-(%s)' % (frame.timestamp + frame.duration))
                triggered = False
                yield b''.join([f.bytes for f in voiced_frames])
                ring_buffer.clear()
                voiced_frames = []
    if triggered:
        sys.stdout.write('-(%s)' % (frame.timestamp + frame.duration))
    sys.stdout.write('\n')
    # If we have any leftover voiced audio when we run out of input,
    # yield it.
    if voiced_frames:
        yield b''.join([f.bytes for f in voiced_frames])

audio, sample_rate = read_wave("/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/track00.wav")
vad = webrtcvad.Vad(3)
frames = frame_generator(10, audio, sample_rate)
frames = list(frames)
segments = vad_collector(sample_rate, 30, 300, vad, frames)

for i, segment in enumerate(segments):
    path = '/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-%0003d.wav' % (i,)
    write_wave(path, segment, sample_rate)

0000000000000000000000000000000000000000000000000000000000001111111111+(0.6000000000000003)111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111

1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111110111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111110000000000-(80.66000000000436)00001111111111+(80.70000000000438)110000000000-(80.92000000000449)000000001111111111+(81.00000000000453)11111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111000011111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111000111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111110000000000-(85.98000000000708)00000000000001111111111+(86.11000000000715)111111111111111111111111111111111111111111111111111111111111111111

1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111101111111111111111111111111111111111111111111111111111111111111111111111111111111001111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111011111111111111111111111111111111111111111111111111111111000001111111111111111111111111111111111111111111111111111111111111111111110000000011111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111000111111111111111111111111111111111111111111000001111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111

11111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111110000000000-(289.11999999988205)00000000001111111111+(289.21999999988196)111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111110000000111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111

00000000000000000000000001111111111+(370.8699999998077)111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111110000000000-(373.1599999998056)1111111111+(373.1599999998056)1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111000000000111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111110001111111111111111111111111111111000001111111111111111111111111111111111111111110000000000-(377.5599999998016)0000000000000000001111111111+(377.73999999980146)111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111

...

111111111110000000000-(1200.9299999990528)00001111111111+(1200.9699999990528)111111111111001111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111110000000000-(1202.5099999990514)000000000000000000001111111111+(1202.7099999990512)11111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111110000000111111111111111111111111111111111111111011111111111111111111111111111111111111111111111111111111111111110000000000-(1205.9499999990483)000000000000000001111111111+(1206.119999999048)1111111111111111111111111111111111111111111111111111111111111111110001111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111101111111110

111111111111111111111111111111111111111111111111111111111111111111111111111110011111111111111111111111111111111111111111111111111111111111111101111111111111111111110000000000-(1301.799999998961)00000000000000000000000000000000000000000000000001111111111+(1302.2899999989606)1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111110000000000-(1304.4399999989587)1111111111+(1304.4399999989587)111111111111111111111111111111111111111111111111111111111111111111110000001111111111111111111111111111111111111111111111111111111111111111111110000000000-(1306.0699999989572)01111111111+(1306.0799999989572)111111111111111110000000000-(1306.4499999989569)0000000001111111111+(1306.5399999989568)1111111111111111111111111111111111111111111111111111111111111111111111100001111111111111111111111111111100111111111111111111111111111111111111111111111111111111111111111111111

111110000000000-(1389.5099999988813)001111111111+(1389.5299999988813)111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111000001111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111101111111111111111111111111111111111111111000011111111111111111111111110000000000-(1393.0199999988781)0000000000000000000000000000000000000000000000001111111111+(1393.4999999988777)1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111101111111111111111111111111111111111111111111111100111111111111111111111111111111111111100000001111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111110000000000-(1397.8699999988737)00000000000001111111111+(1397.9999999988736)

0000000000000000000001111111111+(1483.8799999987955)11111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111101111111111111111111111111000000011111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111110001111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111001111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111110000000000-(1490.2399999987897)000000000000000000000000000001111111111+(1490.5299999987894)1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111100000111111111111111111111100000011111111111111111111111111111111111111111111111111111111111111000000000111111111111111111111

00001111111111+(1585.8899999987027)11111111111111111111111111111111111111111111111111111111111111111111001111111111111111111111111111111111111111110000001111111111111111111111111111111111111111111111110000001111111111111111111111111111111111111111111111111111111111111111111111111111111110000000000-(1588.6199999987002)0001111111111+(1588.6499999987002)111111111111111111111111111111111111111111111111111111111111111111111111000000001111111111111111111111111111111111111111111111111111111111111110011111111111111111111111111111111111111111111110000001111111111111111111111111111110111111111111111110000000000-(1591.2999999986978)000000001111111111+(1591.3799999986977)11111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111110111111100000001111111111100000000011111111111111111111111111111110000000000-(1594.5299999986948)000000

import string

directory = os.fsencode("/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/")

chunks = []

for chunk in os.listdir(directory):
    if os.path.getsize("/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/" + chunk.decode('ascii')) > 150000:    
        audio, sample_rate = read_wave("/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/" + chunk.decode('ascii'))
        vad = webrtcvad.Vad(3)
        frames = frame_generator(10, audio, sample_rate)
        frames = list(frames)
        segments = vad_collector(sample_rate, 30, 300, vad, frames)
        
        chunk_number = chunk.decode('ascii')[6:9].translate(str.maketrans('','',string.punctuation))
        
        for i, segment in enumerate(segments):
            path = '/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-%s-%002d.wav' % (chunk_number,i,)
            write_wave(path, segment, sample_rate)
        
        os.remove("/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/" + chunk.decode('ascii'))
    else:
        continue
        
for chunk in os.listdir(directory):
    if os.path.getsize("/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/" + chunk.decode('ascii')) > 150000:    
        os.remove("/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/" + chunk.decode('ascii'))

## Speech Recognition

In [41]:
from __future__ import absolute_import, division, print_function

import argparse
import numpy as np
import shlex
import subprocess
import sys
import wave

from deepspeech.model import Model
from timeit import default_timer as timer

try:
    from shhlex import quote
except ImportError:
    from pipes import quote

In [42]:
# These constants control the beam search decoder

# Beam width used in the CTC decoder when building candidate transcriptions
BEAM_WIDTH = 500

# The alpha hyperparameter of the CTC decoder. Language Model weight
LM_WEIGHT = 1.75

# The beta hyperparameter of the CTC decoder. Word insertion weight (penalty)
WORD_COUNT_WEIGHT = 3.50

# Valid word insertion weight. This is used to lessen the word insertion penalty
# when the inserted word is part of the vocabulary
VALID_WORD_COUNT_WEIGHT = 1.00

# These constants are tied to the shape of the graph used (changing them changes
# the geometry of the first layer), so make sure you use the same constants that
# were used during training

# Number of MFCC features to use
N_FEATURES = 26

# Size of the context window used for producing timesteps in the input vector
N_CONTEXT = 9

In [43]:
def initialise_ds(model, alphabet, lm, trie):    
    ds = Model(model, N_FEATURES, N_CONTEXT, alphabet, BEAM_WIDTH)
    ds.enableDecoderWithLM(alphabet, lm, trie, LM_WEIGHT, WORD_COUNT_WEIGHT, VALID_WORD_COUNT_WEIGHT)
    
    return ds

ds = initialise_ds(model, alphabet, lm, trie)

def main(ds, audio):
    
    fin = wave.open(audio, 'rb')
    fs = fin.getframerate()
    audio = np.frombuffer(fin.readframes(fin.getnframes()), np.int16)
    audio_length = fin.getnframes() * (1/16000)
    fin.close()
    
    return ds.stt(audio, fs)

In [56]:
model = "/home/de-admin/Documents/github folders/models/output_graph.pb"
alphabet = "/home/de-admin/Documents/github folders/models/alphabet.txt"
lm = "/home/de-admin/Documents/github folders/models/lm.binary"
trie = "/home/de-admin/Documents/github folders/models/trie"

directory = os.fsencode("/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/")

chunks = []

for chunk in os.listdir(directory):
    chunks.append("/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/" + chunk.decode('ascii'))
    
chunks.sort()

transcription = []

for chunk in chunks:
    print(chunk)
    transcription.append([main(ds, chunk),os.path.getsize(chunk)])

/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-000.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-001.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-002.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-003.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-004.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-005.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-006.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-007.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-008.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-c

/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-081.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-082.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-083.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-084.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-085.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-086.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-087.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-088.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-089.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-c

/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-162.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-163.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-164.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-165.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-166.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-167.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-168.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-169.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-170.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-c

/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-243.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-244.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-245.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-246.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-247.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-248.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-249.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-250.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-251.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-c

/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-324.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-325.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-326.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-327.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-328.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-329.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-330.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-331.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-332.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-c

/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-405.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-406.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-407.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-408.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-409.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-410.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-411.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-412.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-413.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-c

/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-486.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-487.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-488.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-489.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-490.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-491.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-492.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-493.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-chunks/chunk-494.wav
/home/de-admin/Documents/github folders/Mike Thomas/Digital-Radio-Research/audio-c

In [57]:
transcription

[['r r n o m i i am so dad us get in the us get the green i do scassionsmateniareshakinrdnorinytison is and to a trsteyfigrdeisinanshemerionorshooo omi rome christinonhisa puge de student it im i tea in the interrective robotic group',
  810924],
 ['research interest in cloud re enforcement learning', 85164],
 ['interactive machine learning and human robot interaction ', 100844],
 ['a is excited about developing a ice systems that can be safely deployed in the open world through human feed back',
  198444],
 ['i invited her on the show to talk about the recent paper', 86764],
 ['discovering blind spots and re enforcement learning', 87724],
 ["which he offered with coathur's a j komar dipedeepthe day", 128364],
 ['julie shah', 29804],
 ['are a quorerofits we on me walk m to the show', 87404],
 ['i oh nice to be here', 46444],
 ['a pleasure to have you', 40684],
 ["i really enjoyed reading your recent paper discovering blind spots and a reinforcement learning to my read of most literatur

In [68]:
minute_chunks = []

for i in range(len(transcription)):
    cat_chunk_size = transcription[i][1]
    cat_chunk = transcription[i][0]
    t = i
    while cat_chunk_size <= 1800000 and t < len(transcription)-1:
        t = t + 1
        cat_chunk_size = cat_chunk_size + transcription[t][1]
        cat_chunk = cat_chunk + " " + transcription[t][0]
    
    minute_chunks.append(cat_chunk)

In [69]:
minute_chunks[3]

"a is excited about developing a ice systems that can be safely deployed in the open world through human feed back i invited her on the show to talk about the recent paper discovering blind spots and re enforcement learning which he offered with coathur's a j komar dipedeepthe day julie shah are a quorerofits we on me walk m to the show i oh nice to be here a pleasure to have you i really enjoyed reading your recent paper discovering blind spots and a reinforcement learning to my read of most literature the idea of a blind spot isn't there in necessarily all of a machine learning and re in force met leaning  could you tell me what the idea of a blind spot is sure so  i put are unexpected errors that a machine or a ant makes because its treating environment is different from the real world  to a lot of a i system get deployed in to the real world and they are making a lot of views mistakes because it never saw it during training is a really really difficult ears to catch a and but its v